In [1]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import pickle
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input

# Load trained model
model = load_model('s2s.h5')

# Load data for encoding and decoding
with open('training_data.pkl', 'rb') as f:
    data = pickle.load(f)

input_characters = data['input_characters']
target_characters = data['target_characters']
max_input_length = data['max_input_length']
max_target_length = data['max_target_length']
num_en_chars = data['num_en_chars']
num_dec_chars = data['num_dec_chars']

def encode_input_text(input_text):
    input_text = input_text.lower()
    input_text = input_text[:max_input_length]  # Truncate if necessary
    en_in_data = np.zeros((1, max_input_length, num_en_chars), dtype='float32')
    for t, char in enumerate(input_text):
        if char in input_characters:
            en_in_data[0, t, input_characters.index(char)] = 1.0
        else:
            print(f"Character '{char}' not in input_characters, ignoring.")
    return en_in_data

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1, num_dec_chars))
    target_seq[0, 0, target_characters.index('\t')] = 1.0
    
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_characters[sampled_token_index]
        decoded_sentence += sampled_char
        
        if sampled_char == '\n' or len(decoded_sentence) > max_target_length:
            stop_condition = True
        
        target_seq = np.zeros((1, 1, num_dec_chars))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    
    return decoded_sentence

# Load the encoder and decoder models separately
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = Input(shape=(256,), name='input_3')
decoder_state_input_c = Input(shape=(256,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

def translate_text():
    input_text = input_entry.get()
    if not input_text:
        messagebox.showwarning("Input Error", "Please enter some text to translate.")
        return
    
    input_seq = encode_input_text(input_text)
    translated_text = decode_sequence(input_seq)
    
    output_label.config(text=f"Translated Text: {translated_text}")

# Create the main window
root = tk.Tk()
root.title("English to French Translator")

# Create and place widgets
input_label = tk.Label(root, text="Enter English Text:")
input_label.pack(pady=5)

input_entry = tk.Entry(root, width=50)
input_entry.pack(pady=5)

translate_button = tk.Button(root, text="Translate", command=translate_text)
translate_button.pack(pady=5)

output_label = tk.Label(root, text="Translated Text:")
output_label.pack(pady=5)

# Run the GUI event loop
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
